## NoobHeart's Pytorch implementation  

PhysBench does not rely on any deep learning framework, it handles all preprocessing, postprocessing, visualization, and then exposes the core model development completely to users who can flexibly choose their own development tools.  

### Preparation  

First, you need to prepare the dataset and training data. Please refer to `Noob Heart.ipynb`, this part is the same.  

In [4]:
import sys
sys.path.append("..")
from utils import * 

# To resolve the conflict between torch and numpy, sometimes it can cause a core dump.
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim

Initialize the convolutional kernel weights of NoobHeart and define the model structure.

In [5]:
class NoobHeart(nn.Module):

    def __init__(self,):
        super().__init__()
        self.conv1 = nn.Conv3d(3, 4, (2, 2, 2), stride=(1, 2, 2))
        self.conv2 = nn.Conv3d(4, 2, (2, 2, 2), stride=(1, 2, 2), padding=[1,0,0,])
        self.conv3 = nn.Conv3d(2, 1, (1, 1, 1), stride=(1, 1, 1))
        self.pool  = nn.AvgPool3d((1, 2, 2))
        self.N = lambda x, ax=2:(x-torch.unsqueeze(x.mean(axis=ax), ax))/torch.unsqueeze(x.std(axis=ax), ax)

    def __call__(self, x):
        x = self.N(x.permute(0, 4, 1, 2, 3))
        x = self.N(torch.tanh(self.conv1(x)))
        x = self.N(torch.tanh(self.conv2(x)))
        x = self.pool(x)
        x = self.conv3(x)
        return x.view(x.shape[0], -1)
    
model = NoobHeart()

Write the training loop

In [6]:
batch = 32
train = load_datatape("train_tape.h5", batch=batch) # Training set
valid = load_datatape("valid_tape.h5", batch=batch) # Validation set

optimizer = optim.Adam(model.parameters())

model.cuda()
best = None
for epoch in range(10): # 10 epochs
    model.train()
    for step, (data, label) in enumerate(train): # train
        data, label = torch.tensor(data).float().cuda(), torch.tensor(label).float().cuda()
        optimizer.zero_grad()
        loss = (model(data)-label).abs().mean()
        loss.backward()
        optimizer.step()
    model.eval()
    vloss = []
    for step, (data, label) in enumerate(valid): # valid
        data, label = torch.tensor(data).float().cuda(), torch.tensor(label).float().cuda()
        vloss.append((model(data)-label).abs().mean())
    vloss = torch.stack(vloss).mean()
    print(f'{epoch=},\tval_loss={vloss}')
    if best is None or best>vloss:
        torch.save(model.state_dict(), 'NoobHeart.pkl')
        print('Best model saved')
        best = vloss

model.load_state_dict(torch.load('NoobHeart.pkl')) # load the best
model_ = lambda x:model(torch.tensor(x.astype(np.float32)).cuda()).detach().cpu().numpy()

epoch=0,	val_loss=0.6574487686157227
Best model saved
epoch=1,	val_loss=0.5800231695175171
Best model saved
epoch=2,	val_loss=0.5603955984115601
Best model saved
epoch=3,	val_loss=0.5514898300170898
Best model saved
epoch=4,	val_loss=0.5484949350357056
Best model saved
epoch=5,	val_loss=0.5488275289535522
epoch=6,	val_loss=0.5489773750305176
epoch=7,	val_loss=0.5491269826889038
epoch=8,	val_loss=0.5497186183929443
epoch=9,	val_loss=0.5492029190063477


Use `eval_on_dataset` to test and obtain metrics.

In [7]:
eval_on_dataset('ubfc_dataset.h5', model_, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_PURE_UBFC.h5')
r = get_metrics('../results/NoobHeart_PURE_UBFC.h5')['Whole video']
print(f'HR metrics: MAE:{r["MAE"]}, RMSE:{r["RMSE"]}, R:{r["R"]}')
r = get_metrics_HRV('../results/NoobHeart_PURE_UBFC.h5')['SDNN']
print(f'HRV metrics: MAE:{r["MAE"]}, RMSE:{r["RMSE"]}, R:{r["R"]}')

  0%|          | 0/42 [00:00<?, ?it/s]

HR metrics: MAE:0.977, RMSE:1.477, R:0.99672
HRV metrics: MAE:35.889, RMSE:40.899, R:0.63317
